## LaRE 

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# imports
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..")))
from core import get_model
# from torchsummary import summary
import torch
# from src.landmark_extraction import extract_rois

In [3]:
model_name = "CLIP:RN50"
model = get_model(model_name)

In [4]:
summary(model.clip_model.visual, (3, 224, 224))

NameError: name 'summary' is not defined

In [14]:
model_name = "CLIP:RN50"
model = get_model(model_name, type="wmap", roi_pooling=False)

### ROI

In [15]:
# Load image and extract landmarks
image = "../assets/john.jpg"
rois = extract_rois(image)
boxes = [box for box, _ in rois]
# Convert to tensor
boxes_tensor = torch.tensor(boxes, dtype=torch.float32)  # shape (N_rois, 4)
# Add batch indices (0 if only 1 image)
batch_indices = torch.zeros((boxes_tensor.shape[0], 1), dtype=torch.float32)
# Concatenate batch index
boxes_tensor = torch.cat([batch_indices, boxes_tensor], dim=1)  # (N_rois, 5)
print(boxes_tensor.shape)  # should be (N_rois, 5)

NameError: name 'extract_rois' is not defined

In [15]:
from PIL import Image
import numpy as np

# Load image
image = Image.open('../assets/john.jpg')
# Resize to 224x224
image = np.array(image.resize((224, 224)))
image_tensor = torch.from_numpy(image).permute(2, 0, 1).float()  # C,H,W
image_tensor /= 255.0
image_tensor = image_tensor[np.newaxis, :]
image_tensor.shape

torch.Size([1, 3, 224, 224])

In [27]:
import torch
from rich.table import Table
from rich.console import Console
from torchinfo import summary

import torch
from rich.table import Table
from rich.console import Console

def display_model_summary(model, input_shape=None, title="Model Summary", device="cpu"):
    """
    Display model parameter summary using rich. Supports multiple inputs.

    :param:
        model: PyTorch model
    :param:
        input_shape: A single shape tuple or list/tuple e.g., (1, 3, 224, 224)
    :param:
        title: Optional table title
    :param:
        device: Device for dummy inputs
    """
    model = model.to(device)
    table = Table(title=title)
    table.add_column("Property", style="cyan", no_wrap=True)
    table.add_column("Value", style="magenta")

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    table.add_row("Model", model.__class__.__name__)
    table.add_row("Total Parameters", f"{total_params:,}")
    table.add_row("Trainable Parameters", f"{trainable_params:,}")
    table.add_row("Trainable (Mi)", f"{trainable_params / (1024 ** 2):.2f} Million")
    table.add_row("Summary", f"{summary(model.clip_model, depth=1)}")

    if input_shape is not None:
        table.add_row(f"Input Shape", str(input_shape))
        table.add_row("CLIP.Visual", 
        f"{summary(model.clip_model.visual, input_size=input_shape, depth=1)}")

    console = Console()
    console.print(table)

display_model_summary(model, input_shape=(1, 3, 224, 224))

                                                   Model Summary                                                   
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                                                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Model                │ CLIPClassifierWMap                                                                       │
│ Total Parameters     │ 104,453,515                                                                              │
│ Trainable Parameters │ 104,453,515                                                                              │
│ Trainable (Mi)       │ 99.61 Million                                                                            │
│ Summary              │ =======================================================================================… │
│                      │ Layer (type:depth-idx)                                            Param #                │
│                      │ =======================================================================================… │
│                      │ CLIP                                                              563,713                │
│                      │ ├─ModifiedResNet: 1-1                                             38,316,896             │
│                      │ ├─Transformer: 1-2                                                37,828,608             │
│                      │ ├─Embedding: 1-3                                                  25,296,896             │
│                      │ ├─LayerNorm: 1-4                                                  1,024                  │
│                      │ =======================================================================================… │
│                      │ Total params: 102,007,137                                                                │
│                      │ Trainable params: 102,007,137                                                            │
│                      │ Non-trainable params: 0                                                                  │
│                      │ =======================================================================================… │
│ Input Shape          │ (1, 3, 224, 224)                                                                         │
│ CLIP.Visual          │ =======================================================================================… │
│                      │ Layer (type:depth-idx)                   Output Shape              Param #               │
│                      │ =======================================================================================… │
│                      │ ModifiedResNet                           [1, 1024]                 --                    │
│                      │ ├─Conv2d: 1-1                            [1, 32, 112, 112]         864                   │
│                      │ ├─BatchNorm2d: 1-2                       [1, 32, 112, 112]         64                    │
│                      │ ├─ReLU: 1-3                              [1, 32, 112, 112]         --                    │
│                      │ ├─Conv2d: 1-4                            [1, 32, 112, 112]         9,216                 │
│                      │ ├─BatchNorm2d: 1-5                       [1, 32, 112, 112]         64                    │
│                      │ ├─ReLU: 1-6                              [1, 32, 112, 112]         --                    │
│                      │ ├─Conv2d: 1-7                            [1, 64, 112, 112]         18,432                │
│                      │ ├─BatchNorm2d: 1-8                       [1, 64, 112, 112]         128                   │
│                      │ ├─ReLU: 1-9                    

In [29]:
x = summary(model.clip_model.visual, input_size=(1, 3, 224, 224), depth=1)
str(x)

'==========================================================================================\nLayer (type:depth-idx)                   Output Shape              Param #\n==========================================================================================\nModifiedResNet                           [1, 1024]                 --\n├─Conv2d: 1-1                            [1, 32, 112, 112]         864\n├─BatchNorm2d: 1-2                       [1, 32, 112, 112]         64\n├─ReLU: 1-3                              [1, 32, 112, 112]         --\n├─Conv2d: 1-4                            [1, 32, 112, 112]         9,216\n├─BatchNorm2d: 1-5                       [1, 32, 112, 112]         64\n├─ReLU: 1-6                              [1, 32, 112, 112]         --\n├─Conv2d: 1-7                            [1, 64, 112, 112]         18,432\n├─BatchNorm2d: 1-8                       [1, 64, 112, 112]         128\n├─ReLU: 1-9                              [1, 64, 112, 112]         --\n├─AvgPool2d: 1-10   

In [18]:
summary(model.clip_model, input_size=(3, 224, 224), depth=2)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [4]:
image_input = torch.randn(1, 3, 224, 224)
loss_map = torch.randn(1, 4, 32, 32)

with torch.no_grad():
    logits = model(image_tensor, loss_map)
    print(logits.shape)

NameError: name 'model' is not defined

In [2]:
import torch 
loss_map_path = "/home/scur0555/udit/lare/test/outputs/heygen/fake_000.pt"
loss_map = torch.load(loss_map_path)
print(type(loss_map))  # Should be <class 'torch.Tensor'> if it was a tensor

<class 'torch.Tensor'>


In [7]:
torch.stack(outs, dim=0).shape

torch.Size([2, 4, 32, 32])

In [5]:
loss_map.shape

torch.Size([4, 32, 32])

In [4]:
outs = [loss_map, loss_map]
outs

[tensor([[[1.2722, 0.6045, 0.1884,  ..., 0.1580, 0.5751, 0.7202],
          [0.5247, 0.1231, 0.0153,  ..., 0.9977, 0.2338, 0.3606],
          [0.9416, 1.0371, 0.1474,  ..., 0.2346, 0.1005, 0.4632],
          ...,
          [0.0609, 0.4053, 0.1464,  ..., 0.1551, 0.2330, 0.2857],
          [0.0782, 0.0202, 0.1628,  ..., 0.1930, 0.1445, 0.6617],
          [0.1446, 0.0791, 0.1277,  ..., 0.0954, 0.1362, 0.5819]],
 
         [[2.0497, 1.5650, 0.4817,  ..., 0.0799, 0.3167, 0.1323],
          [0.3328, 0.5117, 0.0777,  ..., 0.1557, 0.3344, 0.3120],
          [1.3182, 0.4221, 0.1938,  ..., 0.1400, 0.1776, 0.0101],
          ...,
          [0.0423, 0.0564, 0.1431,  ..., 0.8686, 0.2950, 0.7666],
          [0.1798, 0.0861, 0.0021,  ..., 0.3287, 0.1534, 0.6862],
          [0.0841, 0.1421, 0.4049,  ..., 0.1308, 0.2535, 0.2070]],
 
         [[1.0831, 0.4674, 0.3141,  ..., 0.0589, 0.1330, 0.3242],
          [1.2137, 0.8454, 0.3480,  ..., 0.2199, 0.3875, 1.4261],
          [0.8819, 0.3374, 0.5654,  ...,